In [3]:
import os

# Replace 'your_directory_path' with the path to the directory you want to check
root_directory = 'D:\dst\\rgb'

print(f"Contents of {root_directory}:")
print(os.listdir(root_directory))



Contents of D:\dst\rgb:
['palmar_wrist_neutral', 'palmar_wrist_prongated', 'pinch', 'tripod']


In [4]:
import tensorflow as tf
from tensorflow.keras import models,layers

In [5]:
import matplotlib.pyplot as plt

In [6]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
num_classes=4

In [7]:
data = tf.keras.preprocessing.image_dataset_from_directory(
    "D:\dst\\rgb",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 200633 files belonging to 4 classes.


In [8]:
classnames = data.class_names
classnames


['palmar_wrist_neutral', 'palmar_wrist_prongated', 'pinch', 'tripod']

In [35]:
from PIL import Image, UnidentifiedImageError
import os
import time

def load_image(image_path):
    try:
        img = Image.open(image_path)
        return img
    except (UnidentifiedImageError, FileNotFoundError):
        return None

def filter_images(directory):
    valid_images = []
    invalid_images = []
    start_time = time.time()
    
    for root, _, files in os.walk(directory):
        for file in files:
            img = load_image(os.path.join(root, file))
            if img:
                valid_images.append(os.path.join(root, file))
            else:
                invalid_images.append(os.path.join(root, file))

    end_time = time.time()
    total_time = end_time - start_time
    return valid_images, invalid_images, total_time

# Directory path to your dataset
directory = "D:\\dst\\rgb"

# Get a list of valid and invalid images
valid_images, invalid_images, total_time = filter_images(directory)

print(f"Number of valid images: {len(valid_images)}")
print(f"Number of invalid images: {len(invalid_images)}")
print(f"Total time taken: {total_time:.2f} seconds")

# Estimated time for processing 202000 images (assuming the same time per image)
total_images = len(valid_images) + len(invalid_images)
avg_time_per_image = total_time / total_images
estimated_total_time = avg_time_per_image * 202000

print(f"Estimated time for 202000 images: {estimated_total_time:.2f} seconds")


Number of valid images: 200633
Number of invalid images: 1718
Total time taken: 1797.02 seconds
Estimated time for 202000 images: 1793.91 seconds


In [36]:
invalid_images

['D:\\dst\\rgb\\tripod\\cell_phone\\pear\\pear_1\\pear_1_1_100_crop.png',
 'D:\\dst\\rgb\\tripod\\cell_phone\\pear\\pear_1\\pear_1_1_101_crop.png',
 'D:\\dst\\rgb\\tripod\\cell_phone\\pear\\pear_1\\pear_1_1_102_crop.png',
 'D:\\dst\\rgb\\tripod\\cell_phone\\pear\\pear_1\\pear_1_1_103_crop.png',
 'D:\\dst\\rgb\\tripod\\cell_phone\\pear\\pear_1\\pear_1_1_104_crop.png',
 'D:\\dst\\rgb\\tripod\\cell_phone\\pear\\pear_1\\pear_1_1_105_crop.png',
 'D:\\dst\\rgb\\tripod\\cell_phone\\pear\\pear_1\\pear_1_1_106_crop.png',
 'D:\\dst\\rgb\\tripod\\cell_phone\\pear\\pear_1\\pear_1_1_107_crop.png',
 'D:\\dst\\rgb\\tripod\\cell_phone\\pear\\pear_1\\pear_1_1_108_crop.png',
 'D:\\dst\\rgb\\tripod\\cell_phone\\pear\\pear_1\\pear_1_1_109_crop.png',
 'D:\\dst\\rgb\\tripod\\cell_phone\\pear\\pear_1\\pear_1_1_10_crop.png',
 'D:\\dst\\rgb\\tripod\\cell_phone\\pear\\pear_1\\pear_1_1_110_crop.png',
 'D:\\dst\\rgb\\tripod\\cell_phone\\pear\\pear_1\\pear_1_1_111_crop.png',
 'D:\\dst\\rgb\\tripod\\cell_phone\\pea

In [37]:
from PIL import Image, UnidentifiedImageError
import os
import time

def load_image(image_path):
    try:
        img = Image.open(image_path)
        return img
    except (UnidentifiedImageError, FileNotFoundError):
        return None

def delete_invalid_images(directory):
    deleted_count = 0
    for root, _, files in os.walk(directory):
        for file in files:
            img_path = os.path.join(root, file)
            img = load_image(img_path)
            if img is None:
                os.remove(img_path)
                deleted_count += 1
    return deleted_count

# Directory path to your dataset
directory = "D:\\dst\\rgb"

deleted_count = delete_invalid_images(directory)
print(f"Deleted {deleted_count} invalid images.")


Deleted 1718 invalid images.


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Assuming you have defined BATCH_SIZE and num_classes
BATCH_SIZE = 32
num_classes = 4
IMAGE_SIZE = 256

# Create an ImageDataGenerator with rescaling and other augmentation parameters
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Specify the directory path to your dataset
directory = "D:\dst\\rgb"

# Generate data for training and validation sets using the data generator
train_generator = train_datagen.flow_from_directory(
    directory,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Use 'categorical' for multi-class classification
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    directory,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 160507 images belonging to 4 classes.
Found 40126 images belonging to 4 classes.


In [12]:

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Use 'categorical_crossentropy' for multi-class classification
    metrics=['accuracy']
)



In [13]:
# Train the model using the generators
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=1,  # Adjust epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)


  21/5015 [..............................] - ETA: 50:58:41 - loss: 1.5353 - accuracy: 0.3333


KeyboardInterrupt



In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Assuming IMAGE_SIZE and num_classes are defined
IMAGE_SIZE = 256
num_classes = 4

# VGG-like model architecture
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(2, 2)),
    
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(2, 2)),
    
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(2, 2)),
    
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(2, 2)),
    
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 256, 256, 64)      1792      
                                                                 
 conv2d_14 (Conv2D)          (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 128, 128, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 128, 128, 128)     73856     
                                                                 
 conv2d_16 (Conv2D)          (None, 128, 128, 128)     147584    
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 64, 64, 128)       0         
 g2D)                                                 